In [1]:
!pip install boto3

In [2]:
# Initial imports
import os
import io
import json
import numpy as np
import pandas as pd
import boto3

In [3]:
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = "spark-3.2.0"
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graph

In [4]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Pets").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [5]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
file = "amazon_reviews_us_Pet_Products_v1_00.tsv.gz"

from pyspark.sql.types import StringType, IntegerType, DateType, StructType, StructField

spark.sparkContext.addFile(url)
df = spark.read.option("delimiter", "\t").option("header","True").option("inferSchema","True").csv(SparkFiles.get(file))

# Show DataFrame
df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

In [6]:
print(f"Number of rows: {df.count()}")
print(f"Number of distinct rows: {df.distinct().count()}")

Number of rows: 2643619
Number of distinct rows: 2643619


# Cleaning

In [7]:
# Drop duplicate rows - not necessary but included for consistency
df_distinct = df.distinct()
df_distinct.count()

2643619

## Review_id_table

In [8]:
from pyspark.sql.functions import to_date
# Create review_id_table
review_id_table = df_distinct.select("review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date"))
review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R100EO2AQP6B02|   33140435|B00BUQ4AV8|     805940477| 2014-10-12|
|R100IITU077KKZ|   25475634|B00RVY9DRO|     189955978| 2015-06-04|
|R100WIAVSCW4D1|   49229898|B004M8LM44|     419962202| 2013-07-31|
|R1023NXBAJ0BYT|   52227830|B0006IZH68|     470609741| 2004-12-23|
|R102CEVVIBXNFB|   11062585|B0014N70QO|     226028756| 2014-07-14|
|R102MO11LN79HV|   17155412|B0039LFVUG|     517238077| 2010-07-24|
|R102TPQOB6ECI4|   46107297|B008QVKBLS|     212972268| 2015-02-27|
|R1032IWOCCKRV1|   28307907|B00406YHRC|     178991753| 2014-01-08|
|R103QNPJX6OJ6D|   38188681|B003TUQWB4|     287424702| 2014-05-27|
|R104GDOC82SGT4|   49476810|B000YIWUXI|     178342731| 2015-03-13|
|R104OCMU5PF85T|    2654230|B008CO4H8C|     648340511| 2015-03-03|
|R104OIYLZ3E442|   20836291|B000Y91HOU|      17136419| 2014-11

In [9]:
# Check data types
review_id_table.dtypes

[('review_id', 'string'),
 ('customer_id', 'int'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('review_date', 'date')]

## products

In [10]:
products = df_distinct.select("product_id", "product_title")
products.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00BUQ4AV8|Evergreens Pet Do...|
|B00RVY9DRO|Volkman Avian Sci...|
|B004M8LM44|Dogtra YS300 Dog ...|
|B0006IZH68|thedog8it! Gourme...|
|B0014N70QO|Litter Kwitter To...|
|B0039LFVUG|So Bright LLC SO ...|
|B008QVKBLS|Super Bright 3W 2...|
|B00406YHRC|Prevue Pet Produc...|
|B003TUQWB4|Solvit Jumbo Stan...|
|B000YIWUXI|Safari® Cat Self-...|
|B008CO4H8C|Furhaven Hood Pet...|
|B000Y91HOU|Zanies Hard Rubbe...|
|B004HX6QD6|Dogtra IQ Remote ...|
|B00IEVI99G|2 Door Pet Wire C...|
|B004J2BIHO|Top Performance A...|
|B003IUF1FS|FLEA + TICK Repel...|
|B0002APXJQ|           Quick-Net|
|B008CUZJF6|DIY CO2 Aquarium ...|
|B004B3W4N8|6" BULLY STICKS -...|
|B004UMMJC6|Frontline 3-Pack ...|
+----------+--------------------+
only showing top 20 rows



In [11]:
products.dtypes

[('product_id', 'string'), ('product_title', 'string')]

In [12]:
# Drop duplicates to keep only one review per ID
# Loses data, but is needed for upload to table with specified schema
products_dist = products.dropDuplicates(['product_id'])

In [13]:
products_dist.count()

239341

## customers

In [14]:
customers = df_distinct.groupby("customer_id").count()
customers = customers.withColumnRenamed("count","customer_count")
customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   37223012|             5|
|   52055901|             1|
|   16223525|             5|
|   28023775|             3|
|   51092659|             3|
|   44748307|            11|
|   33283869|             1|
|   14670648|            21|
|   21340218|             1|
|   40656071|             2|
|   26298325|             1|
|   26074411|            11|
|   50714860|             1|
|   45117705|            14|
|   43340917|            19|
|   36560790|             2|
|   16785445|             1|
|   38342214|             3|
|   10932658|             2|
|   13976780|             1|
+-----------+--------------+
only showing top 20 rows



In [15]:
customers.dtypes

[('customer_id', 'int'), ('customer_count', 'bigint')]

## vine_table

In [16]:
vine_table = df_distinct.select("review_id", "star_rating", "helpful_votes", "total_votes", "vine")
vine_table.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R100EO2AQP6B02|          3|            2|          4|   N|
|R100IITU077KKZ|          5|            0|          0|   N|
|R100WIAVSCW4D1|          5|            3|          5|   N|
|R1023NXBAJ0BYT|          5|            0|          0|   N|
|R102CEVVIBXNFB|          1|            0|          2|   N|
|R102MO11LN79HV|          5|            0|          0|   N|
|R102TPQOB6ECI4|          3|            0|          0|   N|
|R1032IWOCCKRV1|          1|            2|          2|   N|
|R103QNPJX6OJ6D|          5|            1|          1|   N|
|R104GDOC82SGT4|          5|            0|          0|   N|
|R104OCMU5PF85T|          2|           25|         25|   N|
|R104OIYLZ3E442|          1|            0|          0|   N|
|R1059V5TMDI4OK|          5|          117|        121|   N|
|R1063IMZRT6M5U|          5|            

In [17]:
vine_table.dtypes

[('review_id', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string')]

# Database connection

In [18]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-11-26 21:02:15--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.58MB/s    in 0.2s    

2021-11-26 21:02:16 (4.58 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [19]:
mode = "append"
jdbc_url="jdbc:postgresql://<instance>.rds.amazonaws.com:5432/bigdatachallenge"
config = {"user":"<user>", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}

In [20]:

products_dist.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

In [21]:
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)